## Setup

In [99]:
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm
#from linearmodels import PanelOLS
from linearmodels.panel import RandomEffects, PooledOLS, PanelOLS, FirstDifferenceOLS
import numpy as np

## Explanatory analysis

In [227]:
df = pd.read_stata("assignment_1.dta", convert_dates=False)

In [3]:
df.head(7)

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix,trend
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857,2.0
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732,3.0
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331,4.0
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125,5.0
5,1,86,0.034752,0.326062,0.512324,0.439863,6.90,0.001895,2.385681,29.740980,...,3.006608,-0.051062,0.002048,-0.121867,-0.122245,0.052056,-0.015258,0.055322,0.042765,6.0
6,1,87,0.035604,0.298270,0.527596,0.436170,6.71,0.001828,2.422633,30.993681,...,3.006608,0.024198,-0.089089,0.029374,-0.008431,-0.027923,-0.036189,0.041257,-0.193899,7.0


In [4]:
df[[
    "crmrte", 
    "prbconv", 
    "prbarr", 
    "avgsen", 
    "polpc",
    "density",
    'taxpc', 
    'west', 
    'central', 
    'urban'
]]

,crmrte,prbconv,prbarr,avgsen,polpc,density,taxpc,west,central,urban
0,0.039885,0.402062,0.289696,5.61,0.001787,2.307159,25.697630,0,1,0
1,0.038345,0.433005,0.338111,5.59,0.001767,2.330254,24.874252,0,1,0
2,0.030305,0.525703,0.330449,5.80,0.001836,2.341801,26.451443,0,1,0
3,0.034726,0.604706,0.362525,6.89,0.001886,2.346420,26.842348,0,1,0
4,0.036573,0.578723,0.325395,6.55,0.001924,2.364896,28.140337,0,1,0
...,...,...,...,...,...,...,...,...,...,...
625,0.015575,0.480392,0.226667,7.77,0.001073,0.869048,18.905853,1,0,0
626,0.013662,1.410260,0.204188,10.11,0.001109,0.872024,22.704754,1,0,0
627,0.013086,0.830769,0.180556,5.96,0.001054,0.875000,24.123611,1,0,0
628,0.012874,2.250000,0.112676,7.68,0.001088,0.880952,24.981979,1,0,0


In [228]:
df = df.assign(
    log_crmrte=np.log(df["crmrte"]),
    log_prbconv = np.log(df["prbconv"]),
    log_prbarr = np.log(df["prbarr"]),
    log_avgsen = np.log(df["avgsen"]),
    log_polpc = np.log(df["polpc"]),
    log_density = np.log(df["density"])
)

independent_variables = [column for column in df.columns if column.split("_")[0] == "log" and column != "log_crmrte"] + ['taxpc']
dependent_variable = ["log_crmrte"]
geographical_columns = ["west", "central", "urban"]
index_columns = ['county', 'trend', 'year']

df_final = df[independent_variables+dependent_variable+geographical_columns+index_columns]

In [162]:
df_final = df.dropna()

# Generate dummy variables for years
dummy_vars = pd.get_dummies(df_final["year"].astype(str), prefix="year")
df_final = pd.concat([df_final, dummy_vars], axis=1)
year_dummy_columns = list(dummy_vars.columns)[1:] # year_82 is removed to avoid multicollinearity

dummy_vars




,year_82,year_83,year_84,year_85,year_86,year_87
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,0,0,0,1,0,0
5,0,0,0,0,1,0
...,...,...,...,...,...,...
625,0,1,0,0,0,0
626,0,0,1,0,0,0
627,0,0,0,1,0,0
628,0,0,0,0,1,0


## Estimations

- Pooled OLS

In [229]:
# pooled_ols = ols("log_crmrte ~" + " + ".join(log_variables+geographical_columns+["C(year)"]), data=df_final).fit()
# print(pooled_ols.summary())

pooled_ols_df = df_final.set_index(['county', 'trend'])

pooled_ols = PooledOLS.\
    from_formula("log_crmrte ~ 1 +" + " + ".join(independent_variables+geographical_columns+["C(year)"]), data=pooled_ols_df).\
    fit(cov_type="robust")

print(pooled_ols.summary)



                          PooledOLS Estimation Summary                          
Dep. Variable:             log_crmrte   R-squared:                        0.7784
Estimator:                  PooledOLS   R-squared (Between):              0.8283
No. Observations:                 630   R-squared (Within):               0.2560
Date:                Tue, Mar 14 2023   R-squared (Overall):              0.7784
Time:                        19:30:17   Log-likelihood                   -67.723
Cov. Estimator:                Robust                                           
                                        F-statistic:                      143.78
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(15,614)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             243.02
                            

- Random Effects (with linearmodels)

In [230]:
random_effects_df = df_final.set_index(['county', 'trend'])

re_model = RandomEffects.from_formula(
    "log_crmrte ~ 1 +" + " + ".join(independent_variables+geographical_columns+["C(year)"]),
    random_effects_df
).fit(cov_type="robust") #cov_type="robust"

print(re_model.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:             log_crmrte   R-squared:                        0.5298
Estimator:              RandomEffects   R-squared (Between):              0.7810
No. Observations:                 630   R-squared (Within):               0.4005
Date:                Tue, Mar 14 2023   R-squared (Overall):              0.7478
Time:                        19:30:25   Log-likelihood                    333.01
Cov. Estimator:                Robust                                           
                                        F-statistic:                      46.130
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(15,614)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             38.412
                            

- Fixed Effects (with linearmodels)

In [231]:
# PanelOLS with_formula()
# (we should test time fixed effects by calculating the joint significance of the dummy variables)
# we can't add time-invariant variables, like the geographical ones!

fixed_effects_df = df_final.set_index(['county', 'trend'])

fe_model = PanelOLS.from_formula(
    "log_crmrte ~ 1 + " + " + ".join(independent_variables+["C(year)", "EntityEffects"]), #year_dummy_columns #"C(year)"
    data=fixed_effects_df
).fit(
    cov_type="clustered", cluster_entity=True
)

print(fe_model.summary)

## Example using different method
# exog = sm.add_constant(fixed_effects_df[log_variables+geographical_columns+year_dummy_columns])
# fe_model = PanelOLS(fixed_effects_df.log_crmrte, exog).fit(cov_type="clustered", cluster_entity=True) #cov_type="clustered", cluster_entity=True
# print(fe_model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:             log_crmrte   R-squared:                        0.4039
Estimator:                   PanelOLS   R-squared (Between):              0.5742
No. Observations:                 630   R-squared (Within):               0.4039
Date:                Tue, Mar 14 2023   R-squared (Overall):              0.5594
Time:                        19:30:35   Log-likelihood                    389.11
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      29.808
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(12,528)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             11.035
                            

- Hausman test

Since we don't reject the null hypothesis that the coefficients from both methods are statistically equal, we should use Random Effects instead of Random Effects

In [232]:
# Hausman test of FE vs. RE
# look: http://www.upfie.net/downloads14.html
# It has an Wooldridge exercise solved 

from scipy.stats import chi2

# get the covariance matrices for the two models
b_fe_cov = fe_model.cov
b_re_cov = re_model.cov

# (I) find overlapping coefficients:
common_coef = set(fe_model.params.index).intersection(re_model.params.index)

# (II) calculate differences between FE and RE:
b_diff = np.array(fe_model.params[common_coef] - re_model.params[common_coef])
df = len(b_diff)
b_diff.reshape((df, 1))
b_cov_diff = np.array(b_fe_cov.loc[common_coef, common_coef] -
                      b_re_cov.loc[common_coef, common_coef])
b_cov_diff.reshape((df, df))

# (III) calculate test statistic:
stat = abs(np.transpose(b_diff) @ np.linalg.inv(b_cov_diff) @ b_diff)
pval = 1 - chi2.cdf(stat, df)


print(f'stat: {stat}')
print(f'pval: {pval}')

stat: 23.556729313272996
pval: 0.03546089091857885


- Testing models for serial correlation (Wooldridge, 2022)

In [239]:
independent_variables

['log_prbconv',
 'log_prbarr',
 'log_avgsen',
 'log_polpc',
 'log_density',
 'taxpc']

In [242]:
first_difference_df = df_final.set_index(['county', 'trend'])
exog = first_difference_df[independent_variables]
first_difference_ols = FirstDifferenceOLS(first_difference_df.log_crmrte, exog).fit(cov_type="clustered", cluster_entity=True)

print(first_difference_ols.summary)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:             log_crmrte   R-squared:                        0.3281
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.7715
No. Observations:                 540   R-squared (Within):               0.3041
Date:                Tue, Mar 14 2023   R-squared (Overall):              0.7705
Time:                        21:58:01   Log-likelihood                    202.83
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      43.452
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(6,534)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             8.4744
                            

In [244]:
from statsmodels.stats.diagnostic import acorr_ljungbox

residuals = first_difference_ols.resids

test_results = acorr_ljungbox(residuals, lags=[1])

test_statistic, p_value = test_results

if p_value < 0.05:
    print(f"There is evidence of within-group serial correlation in the residuals. P-value: {p_value}")
else:
    print(f"There is no evidence of within-group serial correlation in the residuals. P-value: {p_value}")

There is evidence of within-group serial correlation in the residuals. P-value: [0.00391233]


/Users/gabrieldiasmp/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:559: FutureWarning: The value returned will change to a single DataFrame after 0.12 is released.  Set return_df to True to use to return a DataFrame now.  Set return_df to False to silence this warning.
  warnings.warn(msg, FutureWarning)


In [235]:
# As we can see below, our independent variables are significant after controlling fixed effects

restrictions = " = ".join(independent_variables) + " = 0"
fe_model.wald_test(formula=restrictions)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 48.4867
P-value: 0.0000
Distributed: chi2(6)
WaldTestStatistic, id: 0x7f8bd8d13d60

- Comparing the three models

In [177]:
from linearmodels.panel import compare

print(compare({"FE": fe_model, "RE": re_model, "Pooled": pooled_ols}))

                            Model Comparison                           
                                    FE                RE         Pooled
-----------------------------------------------------------------------
Dep. Variable               log_crmrte        log_crmrte     log_crmrte
Estimator                     PanelOLS     RandomEffects      PooledOLS
No. Observations                   540               540            540
Cov. Est.                    Clustered            Robust         Robust
R-squared                       0.4104            0.5371         0.7714
R-Squared (Within)              0.4104            0.4047         0.2711
R-Squared (Between)             0.1974            0.7719         0.8147
R-Squared (Overall)             0.2144            0.7427         0.7714
F-statistic                     27.779            43.518         126.52
P-value (F-stat)                0.0000            0.0000         0.0000
=====================     ============   ===============   =====